Imagenet dataset differs from CIFAR10 or MNIST in that it is complex and large. That's why we want to show how to use imagenet dataset to adversarially train a model. For this purpose, we use ResNet50 as the model and FGSM and LOTS as the attack methods. To successfully run this tutorial, you need to download the imagenet dataset and update the path in the code or config file. Both API and command line are provided in this tutorial.

## API

In [9]:
from advsecurenet.utils.adversarial_target_generator import AdversarialTargetGenerator
from advsecurenet.datasets.dataset_factory import DatasetFactory
from advsecurenet.dataloader import DataLoaderFactory
from advsecurenet.shared.types.dataset import DatasetType
from advsecurenet.attacks.fgsm import FGSM
from advsecurenet.attacks.lots import LOTS
from advsecurenet.models.model_factory import ModelFactory
import advsecurenet.shared.types.configs.attack_configs as AttackConfigs
from advsecurenet.shared.types.configs.defense_configs.adversarial_training_config import AdversarialTrainingConfig
from advsecurenet.utils.data import get_subset_data
from advsecurenet.defenses import AdversarialTraining



In [2]:
imagenet_obj = DatasetFactory.create_dataset(DatasetType.IMAGENET)
imagenet_train_path = "/local/scratch/datasets/ImageNet/ILSVRC2012/train"
imagenet_data = imagenet_obj.load_dataset(train=True, root= imagenet_train_path)


In [3]:
# for the simplicity of the example, we will only use 100 images
num_images = 100
imagenet_data = get_subset_data(imagenet_data, num_images)

In [4]:
assert  len(imagenet_data) == num_images

In [5]:
train_loader = DataLoaderFactory.create_dataloader(dataset=imagenet_data, batch_size=32, shuffle=True)

In [6]:
model = ModelFactory.create_model("resnet50", pretrained=False, num_classes=1000)

In [7]:
# get FGSM attack
fgsm_config = AttackConfigs.FgsmAttackConfig(epsilon=0.5, device="cuda:1")
fgsm = FGSM(fgsm_config)

In [8]:
adversarial_training_config = AdversarialTrainingConfig(model=model, models=[model], attacks=[fgsm], train_loader=train_loader, epochs=1, device="cuda:1")
adversarial_training = AdversarialTraining(adversarial_training_config)
adversarial_training.train()

Running epoch 1...


100%|██████████| 4/4 [00:03<00:00,  1.06it/s]

Epoch 1/1 Loss: 8.957191944122314
